## python 敏感词检测

In [16]:
import os
from sklearn.datasets import base
import jieba
import re


stopworddir = "D:\my\ml\data\ch_stopword.txt"

stopwords = [line.strip() for line in open(stopworddir, 'r', encoding='utf-8').readlines()]  

normaldatatxt = "D:\my\ml\data\\centens_word\\晚章.txt"
normaldatadirtxt = "D:\my\ml\data\\centens_word\\正常.txt"
content = ''
with open(normaldatatxt) as f:
    content = f.read()
    content = re.sub("[\s\r\n]","",content)

if len(content)>0:
    words = jieba.cut(content)

    change_list = []
    for word in words:
        if word not in stopwords:
            change_list.append(word)
    change_str = "\n".join(list(set(change_list)))
    print(len(change_str))
    with open(normaldatadirtxt,"w+") as f:
        f.write(change_str)


45603


In [67]:
import random
import os
from sklearn.datasets import base
import jieba

def loaddata():
    stopworddir = "D:\my\ml\data\centens_word"

    filenames = os.listdir(stopworddir)
    ## 获取所有分类
    train_data = base.Bunch(label=[],contents=[],tfidf=[])

    test_data = base.Bunch(label=[],contents=[],tfidf=[])

    for filename in filenames:
        
        with open(os.path.join(stopworddir,filename)) as f:
            content = f.read()
            content = content.replace("\r","\n")
            content = content.replace("\n\n","\n")
            lines = content.split("\n")
            n = 0
            l = len(lines)
            ## 分类不平衡，对数据少的类别进行重复采样
            while (l<14000 and n<10) or l>14000:
                extendlabel = [filename]*l
                random.shuffle(lines)
                train_data.contents.extend(lines)
                train_data.label.extend(extendlabel)
                n = n+1
                if l>14000:
                    break
                
    return train_data
    

train_data = loaddata()

In [69]:
print(len(train_data.label))
print(len(train_data.contents))

38680
38680


In [70]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

import numpy as np
f = CountVectorizer()
X = f.fit_transform(train_data.contents)


X_train, X_test, y_train, y_test = train_test_split(X, train_data.label, test_size=0.33, random_state=42)
print([np.shape(X_train), np.shape(X_test)])

[(25915, 14992), (12765, 14992)]


In [71]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
alpha_list = [0.00001,0.1,1,2,4]

for al in alpha_list:
    clf = MultinomialNB(alpha=al)                                  
    accs=cross_val_score(clf,X_train ,y_train,cv=8,scoring='accuracy')
    print('交叉验证结果:',accs)

交叉验证结果: [0.98920419 0.99167181 0.98581123 0.99012041 0.98981167 0.99166152
 0.99382144 0.98640297]
交叉验证结果: [0.98766194 0.99167181 0.98735349 0.98950293 0.99012041 0.99104385
 0.99382144 0.98702101]
交叉验证结果: [0.9605182  0.96360271 0.96082665 0.95986416 0.96326027 0.95985176
 0.96200185 0.96260816]
交叉验证结果: [0.89265885 0.8923504  0.89481801 0.89286817 0.89626428 0.89314392
 0.89620019 0.89091471]
交叉验证结果: [0.76465145 0.75663171 0.7553979  0.76072862 0.76134609 0.75200741
 0.76367006 0.74814586]


In [82]:
clf = MultinomialNB(alpha=0.00001)
clf.fit(X_train,y_train)


test_predict = clf.predict(X_test)


print(test_predict.shape)
count = 0
for flabel,expct_cate in zip(y_test,test_predict[:]):
    if flabel != expct_cate:
        count = count+1

print("预测结束 测试集总共%d ，分类错误 %d" % (len(y_test),count))

(12765,)
预测结束 测试集总共12765 ，分类错误 131


In [83]:
print(X_test.shape)

(12765, 14992)


In [85]:
teststr = jieba.cut("全球化进程加快，盗窃猖獗")
words = list(teststr)
x_teststr = f.transform(words)
print(clf.predict(x_teststr))
print(words)

['正常.txt' '正常.txt' '正常.txt' '正常.txt' '民生词库.txt' '正常.txt']
['全球化', '进程', '加快', '，', '盗窃', '猖獗']
